In [6]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tf_keras

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 2.2 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/376.0 MB 5.6 MB/s eta 0:01:07
   ---------------------------------------- 1.6/376.0 MB 4.4 MB/s eta 0:01:25
   ---------------------------------------- 2.6/376.0 MB 4.6 MB/s eta 0:01:22
   ---------------------------------------- 3.7/376.0 MB 4.4 MB/s eta 0:01:26
   ---------------------------------------- 4.5/376.0 MB 4.3 MB/s eta 0:01:28
    --------------------------------------- 5.2/376.0 MB 4.2 MB/s eta 0:01:28
    -------------

  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\hp\\AppData\\Roaming\\Python\\Python312\\site-packages\\tensorflow\\compiler\\mlir\\lite\\python\\_pywrap_converter_api.pyd'
Check the permissions.



In [2]:
!pip install transformers tensorflow datasets gradio scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.4 MB 3.7 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/10.4 MB 3.7 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.4 MB 3.9 MB/s eta 0:00:02
   --------------- ------------------------ 3.9/10.4 MB 4.0 MB/s eta 0:00:02
   ------------------- -------------------- 5.0/10.4 MB 4.2 MB/s eta 0:00:02
   ----------------------- ---------------- 6.0/10.4 MB 4.2 MB/s eta 0:00:02
   --------------------------- ------------ 7.1/10.4 MB 4.3 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.4 MB 4.4 MB/s eta 0:00:01
   ----------------------------------- ---- 9.2/10.4 MB 4.4 MB/s eta 0:00:01
   -------------------------------------- - 10.0/10.4 MB 4.3 MB/s eta 0:00:01
   -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import load_dataset
import numpy as np
import gradio as gr
from sklearn.metrics import classification_report

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
# Load dataset
dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="tf"
    )

# Tokenize datasets
train_tokenized = train_data.map(tokenize_function, batched=True)
test_tokenized = test_data.map(tokenize_function, batched=True)

In [ ]:
tf_train_dataset = train_tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask", "label"],
    label_cols=["label"],
    shuffle=True,
    batch_size=16
)

tf_test_dataset = test_tokenized.to_tf_dataset(
    columns=["input_ids", "attention_mask", "label"],
    label_cols=["label"],
    shuffle=False,
    batch_size=16
)

In [ ]:
# Load BERT model with class weighting
model = TFBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    id2label={0: "Negative", 1: "Positive"}
)

# Custom optimizer with weight decay
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    tf_train_dataset,
    validation_data=tf_test_dataset,
    epochs=4,
    callbacks=[early_stopping]
)

In [ ]:
y_true = test_data["label"]
y_pred = model.predict(tf_test_dataset)["logits"]
y_pred = tf.argmax(y_pred, axis=1).numpy()

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Negative", "Positive"]))

In [ ]:
def predict_sentiment(text):
    # Preprocess input
    inputs = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="tf"
    )
    
    # Make prediction
    outputs = model(inputs)
    probs = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
    
    return {
        "Negative": float(probs[0]),
        "Positive": float(probs[1])
    }

# Create Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=3, placeholder="Enter movie review..."),
    outputs=gr.Label(num_top_classes=2),
    examples=[
        ["This movie was an absolute masterpiece! The cinematography was breathtaking."],
        ["A complete waste of time. Poor plot and terrible acting throughout."],
        ["The film had potential but ultimately failed to deliver on its promises."]
    ],
    title="BERT Sentiment Analyzer",
    description="Enter text to analyze sentiment (Negative/Positive) with confidence scores",
    theme="default"
)

# Launch the interface
interface.launch()